In [16]:
import os
import json
import pandas as pd
import traceback
from langchain_openai import AzureOpenAI
from langchain.chat_models import azure_openai
from dotenv import load_dotenv

In [20]:
load_dotenv()

True

In [21]:
DEPLOYMENT_NAME =os.getenv("deployment_name")
API_KEY=os.getenv("api_key")
API_VERSION=os.getenv("api_version")
ENDPOINT=os.getenv("azure_endpoint")

In [22]:
print(API_KEY)

1ebda9eb5627499591c8b3521f94e5eb


In [23]:
llm = AzureOpenAI(
    deployment_name=DEPLOYMENT_NAME,
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT,
    temperature=0.5
)